In [10]:
import os
import re
from io import BytesIO

import fitz
import pdfplumber
from pprint import pprint
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from oauth2client.service_account import ServiceAccountCredentials

from tqdm import tqdm

from multiprocessing import Pool, cpu_count

import logging
from rich.logging import RichHandler

import glob

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 15 ms (started: 2024-02-26 15:44:30 -08:00)


In [22]:
class Path:
    current_dir = os.getcwd()
    project_dir = os.path.dirname(current_dir)

    root_dir = os.path.join(project_dir, "oil-wells-data-scraper")
    gcp_service_account_key = os.path.join(root_dir, "test-5681a-202ae82505a3.json")

    data_text_dir = os.path.join(project_dir, "data", "raw-text")

def get_logger(name):
    # Create a logger
    logger: logging.getLogger(name)

    # Set the logging level (adjust as needed)
    logger.setLevel(logging.DEBUG)

    # Create a console handler and set the level
    ch: RichHandler()
    ch.setLevel(logging.DEBUG)

    # Create a formatter and add it to the handler
    formatter: logging.Formatter("%(message)s")
    ch.setFormatter(formatter)

    # Add the handler to the logger
    logger.addHandler(ch)
    return logger


logger = get_logger("Extracter")

time: 0 ns (started: 2024-02-26 16:15:01 -08:00)


In [3]:
# Authenticate using the application default credentials generated by gcloud
gauth = GoogleAuth()
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    Path.gcp_service_account_key, scopes=["https://www.googleapis.com/auth/drive"]
)
gauth.credentials = credentials
gauth.ServiceAuth()

time: 16 ms (started: 2024-02-26 15:39:12 -08:00)


In [4]:
# Initialize GoogleDrive object without authentication
drive = GoogleDrive(gauth)

# Folder ID from the shared link
folder_id = "12g-bhOylyaMoLF5djocnAeZHBx-gsxgY"

# List all files in the folder
file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()
file_list = [{'id': file_dict['id'], 'title': file_dict['title']} for file_dict in file_list]
file_list

[{'id': '1qXERbF2EhVHPYwPKL0VABomrZ-_n19xY', 'title': 'W28654.pdf'},
 {'id': '1NB3j4hqiZXCxoRTbEnbQFUoRw43DEUdv', 'title': 'W28755.pdf'},
 {'id': '1fgDzVzyPPcXMwqOCDd0-JRtoPOrolciX', 'title': 'W28651.pdf'},
 {'id': '1nz8wvg8mI3LT27VaMZGiFBsQATOSaOf8', 'title': 'W30789.pdf'},
 {'id': '1QThcZsvItpIH6qaBe0pHhT7Qa7uAkyRN', 'title': 'W28652.pdf'},
 {'id': '1UUtKvq0_crRehsXpwI9hTVEcDknlTZAm', 'title': 'W28554.pdf'},
 {'id': '1g85d9r-pEyachdN6dy7H_aRC6Lf4LZJ6', 'title': 'W29242.pdf'},
 {'id': '1ZaDFnuv1gf0arO1b84qYSmqJzdYhP2jJ', 'title': 'W23230.pdf'},
 {'id': '1KgiXY33Fi9oNrYrBTHhRoDgqtsBPQTuw', 'title': 'W28633.pdf'},
 {'id': '19EoftY50DtmIF-MpUyLfFafr7C4UBIz2', 'title': 'W23362.pdf'},
 {'id': '1DuoQUUml5viE9eXDgcjGSNeHP-KFLvmX', 'title': 'W29334.pdf'},
 {'id': '1hFW-vSM7us4hEdZ-CgbhMKROj8AZRSYG', 'title': 'W28754.pdf'},
 {'id': '12akatIC0mO7P61MTBbyXbzmDE5wv2vU2', 'title': 'W28658.pdf'},
 {'id': '1wjjtd6VFaFsEnUMrCHV2vhXBn-rt9Exz', 'title': 'W20197.pdf'},
 {'id': '1-DpH3iXsjsbmYN7Ja99ylFKf

time: 875 ms (started: 2024-02-26 15:39:13 -08:00)


In [32]:
def process_pdf_dep(file, drive):
    filename = file["title"].split(".")[0]
    filepath = os.path.join(Path.data_text_dir, f"{filename}.txt")

    if file["title"].endswith(".pdf"):
        pdf_file = drive.CreateFile({"id": file["id"]})
        print(f"Downloading '{file['title']}' to buffer memory...")
        pdf_bytes = pdf_file.GetContentIOBuffer().read()
        
        # Open the PDF file using pdfplumber
        # with pdfplumber.open(BytesIO(pdf_bytes)) as pdf:
        #     text = ""
        #     for page in pdf.pages:
        #         text += page.extract_text()

        # with open(filepath, "w", encoding="utf-8") as f:
        #     f.write(text)

        print(f"Processing PDF '{file['title']}' with PyMuPDF...")
        # Open file with PyMuPDF
        with fitz.open(stream=BytesIO(pdf_bytes), filetype="pdf") as pdf:
            text = ""
            for page_number in range(pdf.page_count):
                page = pdf.load_page(page_number)
                text += page.get_text()

        with open(filepath, "a", encoding="utf-8") as f:
            f.write(text)
            
        print(f"Saved extract text to '{os.path.relpath(filepath)}'.")


def process_pdf(file, drive):
    filename = file["title"].split(".")[0]
    filepath = os.path.join(Path.data_text_dir, f"{filename}.txt")

    if not os.path.exists(filepath):
        if file["title"].endswith(".pdf"):
            pdf_file = drive.CreateFile({"id": file["id"]})
            logger.info(f"Downloading '{file['title']}' to buffer memory...")
            pdf_bytes = pdf_file.GetContentIOBuffer().read()

            logger.info(f"Processing PDF '{file['title']}' with pdfplumber...")
            # Open the PDF file using pdfplumber
            with pdfplumber.open(BytesIO(pdf_bytes)) as pdf:
                text = ""
                for page in pdf.pages:
                    text += page.extract_text()

            with open(filepath, "w", encoding="utf-8") as f:
                f.write(text)

            logger.info(f"Processing PDF '{file['title']}' with PyMuPDF...")
            # Open file with PyMuPDF
            with fitz.open(stream=BytesIO(pdf_bytes), filetype="pdf") as pdf:
                text = ""
                for page_number in range(pdf.page_count):
                    page = pdf.load_page(page_number)
                    text += page.get_text()

            with open(filepath, "a", encoding="utf-8") as f:
                f.write(text)

            logger.info(f"Saved extract text to '{os.path.relpath(filepath)}'.")
    logger.info(f"'{os.path.relpath(filepath)}' already exists.")

time: 0 ns (started: 2024-02-26 16:24:01 -08:00)


In [7]:
for fileitem in tqdm(file_list):
    process_pdf(fileitem, drive)

100%|██████████| 77/77 [00:00<00:00, 3079.75it/s]

'..\data\raw-text\W28654.txt' already exists.
'..\data\raw-text\W28755.txt' already exists.
'..\data\raw-text\W28651.txt' already exists.
'..\data\raw-text\W30789.txt' already exists.
'..\data\raw-text\W28652.txt' already exists.
'..\data\raw-text\W28554.txt' already exists.
'..\data\raw-text\W29242.txt' already exists.
'..\data\raw-text\W23230.txt' already exists.
'..\data\raw-text\W28633.txt' already exists.
'..\data\raw-text\W23362.txt' already exists.
'..\data\raw-text\W29334.txt' already exists.
'..\data\raw-text\W28754.txt' already exists.
'..\data\raw-text\W28658.txt' already exists.
'..\data\raw-text\W20197.txt' already exists.
'..\data\raw-text\W25158.txt' already exists.
'..\data\raw-text\W29244.txt' already exists.
'..\data\raw-text\W28634.txt' already exists.
'..\data\raw-text\W28648.txt' already exists.
'..\data\raw-text\W20407.txt' already exists.
'..\data\raw-text\W28655.txt' already exists.
'..\data\raw-text\W28744.txt' already exists.
'..\data\raw-text\W28557.txt' alre

In [33]:
for fileitem in tqdm(file_list):
    process_pdf_dep(fileitem, drive)

  0%|          | 0/77 [00:00<?, ?it/s]

Processing PDF 'W28654.pdf' with PyMuPDF...


  1%|▏         | 1/77 [00:10<13:03, 10.31s/it]

Saved extract text to '..\data\raw-text\W28654.txt'.
Processing PDF 'W28755.pdf' with PyMuPDF...


  3%|▎         | 2/77 [00:19<11:42,  9.37s/it]

...

Saved extract text to '..\data\raw-text\W22249.txt'.
Processing PDF 'W25571.pdf' with PyMuPDF...


 99%|█████████▊| 76/77 [06:50<00:03,  3.80s/it]

Saved extract text to '..\data\raw-text\W25571.txt'.


100%|██████████| 77/77 [06:52<00:00,  5.36s/it]

Processing PDF 'W11920.pdf' with PyMuPDF...
Saved extract text to '..\data\raw-text\W11920.txt'.
time: 6min 52s (started: 2024-02-26 16:24:07 -08:00)


In [79]:
text_file_paths = glob.glob(os.path.join(Path.data_text_dir, "*.txt"))

for path in text_file_paths:
    with open(path, "r", encoding="utf-8") as f:
        text = f.read()

    # Print or process the extracted text
    api_id_pattern = (
        r"(?:\b\d{2}-\d{3}-\d{5}\b)|(?:\b\d{2} - \d{3} - \d{5}\b)|(?:^API(?:\s)?#:(?:\s)?\d{10}$)|(?:API(?:\s+)?\d{10})"
    )

    api_id = set(re.findall(api_id_pattern, text))
    api_id = [re.sub(r"[^\d-]", "", id) for id in api_id]

    api_id_final_pattern = r"(\d{2})(\d{3})(\d{5})"
    for i in range(len(api_id)):
        # Find all matches of the pattern in the string
        if len(api_id[i]) != 12:
            match = re.search(api_id_final_pattern, api_id[i])
            if match:
                # Format the match into 'XX-XXX-XXXXX' format
                api_id[i] = f"{match.group(1)}-{match.group(2)}-{match.group(3)}"

    patterns = {
        "operator": r"Well Operator : (.*?)\n",
        "well_name": r"Well Name\s*:\s*(.*)\n",
        "enseco_job": r"\bJob (\d+)\b",
        "job_type": r"Type of Incident : (.*?)\n",
        "county": r"County : (.*?)\n",
        "latitude": r'(\d+°\d+\'\d+\.\d+\"[NS])',
        "longitude": r'(\d+°\d+\'\d+\.\d+\"[EW])',
        "datum": r"Vertical Datum to DDZ\s+([\d.]+ ft)",
        "date_simulated": r"Date Stimulated\s*\n\s*(\d{1,2}/\d{1,2}/\d{4})",
        "formation": r"Stimulated Formation\s*\n\s*([^\n]+)",
        "top_bottom_stimulation_stages": r"Top \(Ft\)\s*Bottom \(Ft\)\s*Stimulation Stages\n\s*(\d+)\s+(\d+)\s+(\d+)",
        "psi": r"Maximum Treatment Pressure \(PSI\)\s*\n\s*(\d+)",
        "lbs": r"Lbs Proppant\s*\n\s*(\d+)",
        "type_treatment": r"Type Treatment\s*\n\s*([^\n]+)",
        "volume": r"Volume Units\s*\n(\d+)\s*(\w+)",
        "max_treatment_rate": r"Maximum Treatment Rate \(BBLS/Min\)\s*\n\s*(\d+(?:\.\d+)?)"
    }

    results = {
        "file_name": os.path.basename(path),
        "id": set(api_id)
    }
    keys_encountered = set()
    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        results[key] = ""
        if match:
            if key not in keys_encountered:
                results[key] = match.group(1).strip()

    print(results)

{'file_name': 'W11745.txt', 'id': set(), 'operator': 'ST MARY LAND & EXPLORATION COMPANY', 'well_name': '', 'enseco_job': '', 'job_type': '', 'county': 'MCKENZIE', 'latitude': '', 'longitude': '', 'datum': '', 'date_simulated': '', 'formation': '', 'top_bottom_stimulation_stages': '', 'psi': '', 'lbs': '', 'type_treatment': '', 'volume': '', 'max_treatment_rate': ''}
{'file_name': 'W11920.txt', 'id': set(), 'operator': 'RIM OPERATING, INC.', 'well_name': '', 'enseco_job': '', 'job_type': 'Valve/Piping Connection Leak', 'county': 'MCKENZIE', 'latitude': '', 'longitude': '', 'datum': '', 'date_simulated': '', 'formation': '', 'top_bottom_stimulation_stages': '', 'psi': '', 'lbs': '', 'type_treatment': '', 'volume': '', 'max_treatment_rate': ''}
{'file_name': 'W15358.txt', 'id': set(), 'operator': 'NANCE PETROLEUM CORPORATION', 'well_name': '', 'enseco_job': '', 'job_type': '', 'county': 'McKenzie', 'latitude': '', 'longitude': '', 'datum': '', 'date_simulated': '', 'formation': '', 'top_

In [ ]:
{'file_name': 'W28654.pdf', 'id': {'33-053-06028'}}
{'file_name': 'W28755.pdf', 'id': {'33-053-03426', '33-053-06056'}}
{'file_name': 'W28651.pdf', 'id': {'33-053-06025'}}
{'file_name': 'W30789.pdf', 'id': {'33-053-06755', '33-053-03426'}}
{'file_name': 'W28652.pdf', 'id': {'33-053-06026'}}
{'file_name': 'W28554.pdf', 'id': {'33-053-05954', '33-053-05998', '33-053-05943', '33-053-05995', '33-053-05997'}}
{'file_name': 'W29242.pdf', 'id': {'33-053-06223'}}
{'file_name': 'W23230.pdf', 'id': {'33-053-03426', '33-053-04211'}}
{'file_name': 'W28633.pdf', 'id': {'33-053-06018'}}
{'file_name': 'W23362.pdf', 'id': {'33-105-02722'}}
{'file_name': 'W29334.pdf', 'id': {'33-053-06243'}}
{'file_name': 'W28754.pdf', 'id': {'33-053-03426', '33-053-06055'}}
{'file_name': 'W28658.pdf', 'id': {'33-053-06030'}}
{'file_name': 'W20197.pdf', 'id': {'33-053-03413'}}
{'file_name': 'W25158.pdf', 'id': {'33-053-04854'}}
{'file_name': 'W29244.pdf', 'id': {'33-053-06225'}}
{'file_name': 'W28634.pdf', 'id': {'33-053-06019'}}
{'file_name': 'W28648.pdf', 'id': {'33-053-06023'}}
{'file_name': 'W20407.pdf', 'id': set()}
{'file_name': 'W28655.pdf', 'id': {'33-053-06029'}}
{'file_name': 'W28744.pdf', 'id': {'33-053-06051'}}
{'file_name': 'W28557.pdf', 'id': {'33-053-05954', '33-053-05998', '33-053-05943', '33-053-05995', '33-053-05997'}}
{'file_name': 'W28649.pdf', 'id': {'33-053-06024'}}
{'file_name': 'W28978.pdf', 'id': {'33-053-06131'}}
{'file_name': 'W28394.pdf', 'id': {'33-053-05954', '33-053-05998', '33-053-05943', '33-053-05995', '33-053-05997'}}
{'file_name': 'W28637.pdf', 'id': {'33-053-06022'}}
{'file_name': 'W28636.pdf', 'id': {'33-053-06021'}}
{'file_name': 'W30188.pdf', 'id': {'33-053-06548'}}
{'file_name': 'W28303.pdf', 'id': {'33-053-05906'}}
{'file_name': 'W23370.pdf', 'id': {'33-105-02730'}}
{'file_name': 'W28425.pdf', 'id': {'33-053-05954', '33-053-05998', '33-053-05943', '33-053-05995', '33-053-05997'}}
{'file_name': 'W28976.pdf', 'id': {'33-053-06129'}}
{'file_name': 'W23367.pdf', 'id': {'33-105-02727'}}
{'file_name': 'W28194.pdf', 'id': {'33-053-05849', '33-053-03433'}}
{'file_name': 'W30189.pdf', 'id': {'33-053-06549'}}
{'file_name': 'W23360.pdf', 'id': {'33-105-02720'}}
{'file_name': 'W90244.pdf', 'id': {'33-053-90244'}}
{'file_name': 'W23366.pdf', 'id': {'33-105-02726'}}
{'file_name': 'W23359.pdf', 'id': {'33-105-02719'}}
{'file_name': 'W36047.pdf', 'id': {'33-053-03433', '33-053-08946'}}
{'file_name': 'W23364.pdf', 'id': {'33-105-02724'}}
{'file_name': 'W29316.pdf', 'id': {'33-053-06231'}}
{'file_name': 'W23372.pdf', 'id': {'33-105-02732'}}
{'file_name': 'W23361.pdf', 'id': {'33-105-02721'}}
{'file_name': 'W23368.pdf', 'id': {'33-105-02728'}}
{'file_name': 'W25157.pdf', 'id': {'33-053-04853'}}
{'file_name': 'W28190.pdf', 'id': {'33-053-05845'}}
{'file_name': 'W15358.pdf', 'id': set()}
{'file_name': 'W29317.pdf', 'id': {'33-053-06232'}}
{'file_name': 'W22099.pdf', 'id': {'33-053-04071', '33-053-03609', '33-053-03936', '33-053-03937', '33-053-03912', '33-053-03911', '33-053-04981'}}
{'file_name': 'W28600.pdf', 'id': {'33-053-06011'}}
{'file_name': 'W23371.pdf', 'id': {'33-105-02731'}}
{'file_name': 'W23363.pdf', 'id': {'33-105-02723'}}
{'file_name': 'W25159.pdf', 'id': {'33-053-04855'}}
{'file_name': 'W25160.pdf', 'id': {'33-053-04856'}}
{'file_name': 'W28756.pdf', 'id': {'33-053-03426', '33-053-06057'}}
{'file_name': 'W90329.pdf', 'id': {'33-053-90329'}}
{'file_name': 'W28342.pdf', 'id': {'33-053-05924'}}
{'file_name': 'W23365.pdf', 'id': {'33-105-02725'}}
{'file_name': 'W11745.pdf', 'id': set()}
{'file_name': 'W25156.pdf', 'id': {'33-053-04852'}}
{'file_name': 'W22221.pdf', 'id': {'33-053-04071', '33-053-03609', '33-053-03936', '33-053-03937', '33-053-03912', '33-053-03911', '33-053-04981'}}
{'file_name': 'W90258.pdf', 'id': set()}
{'file_name': 'W23369.pdf', 'id': {'33-105-02729'}}
{'file_name': 'W22740.pdf', 'id': {'33-053-04071', '33-053-03609', '33-053-03936', '33-053-03937', '33-053-03912', '33-053-03911', '33-053-04981'}}
{'file_name': 'W22220.pdf', 'id': {'33-053-04071', '33-053-03609', '33-053-03936', '33-053-03937', '33-053-03912', '33-053-03911', '33-053-04981'}}
{'file_name': 'W21796.pdf', 'id': set()}
{'file_name': 'W28599.pdf', 'id': {'33-053-06010'}}
{'file_name': 'W22731.pdf', 'id': {'33-053-04069'}}
{'file_name': 'W22247.pdf', 'id': {'33-053-03943'}}
{'file_name': 'W20863.pdf', 'id': set()}
{'file_name': 'W21266.pdf', 'id': set()}
{'file_name': 'W20864.pdf', 'id': {'33-053-04071', '33-053-03609', '33-053-03936', '33-053-03937', '33-053-03912', '33-053-03911', '33-053-04981'}}
{'file_name': 'W28601.pdf', 'id': {'33-053-06012'}}
{'file_name': 'W22249.pdf', 'id': {'33-053-03944'}}
{'file_name': 'W25571.pdf', 'id': {'33-053-04071', '33-053-03609', '33-053-03936', '33-053-03937', '33-053-03912', '33-053-03911', '33-053-04981'}}
{'file_name': 'W11920.pdf', 'id': set()}

In [14]:
import re

# Given list of strings containing API IDs
api_ids = ['API 3305305924', 'API 3305306010', 'API 3305306011', 'API 3300530347']

# Regular expression pattern to extract the 10-digit numbers after 'API'
api_id_pattern = r'API\s*(\d{2})(\d{3})(\d{5})'

# List to store formatted API IDs
formatted_api_ids = []

# Iterate over each string in the list
for api_string in api_ids:
    # Find all matches of the pattern in the string
    match = re.search(api_id_pattern, api_string)
    if match:
        # Format the match into 'XX-XXX-XXXXX' format
        formatted_api_id = f"{match.group(1)}-{match.group(2)}-{match.group(3)}"
        # Append the formatted API ID to the list
        formatted_api_ids.append(formatted_api_id)

# Print the formatted API IDs
print(formatted_api_ids)


['33-053-05924', '33-053-06010', '33-053-06011', '33-005-30347']
time: 0 ns (started: 2024-02-26 02:45:57 -08:00)


In [21]:
pdf_file = drive.CreateFile({"id": "1GoN9tFDGa7nXXeRA-PySoJUkntKVILKl"})
# print(f"Downloading '{file['title']}' to buffer memory...")
pdf_bytes = pdf_file.GetContentIOBuffer().read()

with pdfplumber.open(BytesIO(pdf_bytes)) as pdf:
    text = ""
    for page in pdf.pages:
        text += page.extract_text()

print(text)

Slawson Exploration Company, Inc.
McKenzie County, ND
Sec. 36-T153N-R101W
Magnum 1-36-25H
Plan A
Survey: Sperry Survey
Standard Report
22 March, 2012
Well Coordinates: 389,408.89 N, 1,204,680.91 E (48° 01' 30.35" N, 103° 37' 20.35" W)
Ground Level: 2,187.00 ft
Local Coordinate Origin: Centered on Well Magnum 1-36-25H
Viewing Datum: RKB 22.00' @ 2209.00ft (Nabors 419)
TVDs to System: N
North Reference: True
Unit System: API - US Survey Feet - Custom
Geodetic Scale Factor Applied
Version: 2003.16 Build: 43ISlawson Exploration Company, Inc.
McKenzie County, ND
Survey Report for Magnum 1-36-25H - Sperry Survey
Measured Vertical Vertical Dogleg
Depth Inclination Azimuth Depth +N/-S +E/-W Section Rate
(ft) (°) (°) (ft) (ft) (ft) (ft) (°/100ft)
9,506.00 0.44 263.55 9,505.25 10.02 23.24 11.31 0.00
Tie On to Extreme Surveys
9,541.00 0.46 293.91 9,540.25 10.06 22.98 11.34 0.68
First Sperry MWD Survey
9,636.00 0.13 208.26 9,635.24 10.12 22.58 11.38 0.49
9,732.00 0.62 275.00 9,731.24 10.07 22.01 1

In [77]:
v = {"a": 1, "b": 2}

set(v.keys())

{'a', 'b'}

time: 16 ms (started: 2024-02-26 19:54:22 -08:00)
